In [3]:
# NEEDED PACKAGES
!pip install --upgrade transformers==4.30.2 tqdm==4.64.1
!pip install --upgrade \
  torch torchvision torchaudio \
  transformers \
  pytorch-pretrained-bert \
  scikit-learn \
  pandas \
  tqdm \
  dotmap \
  matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [1]:
# Mount to your drive so you can persist data and find the repo in your drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=False)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Original cloning
#!git clone https://github.com/zdy93/FTL-Trans.git
%cd /content/drive/MyDrive/CS598-DL-Healthcare/FTL-Trans

/content/drive/MyDrive/CS598-DL-Healthcare/FTL-Trans


In [7]:
# This piece pre-processes your raw data file created by merging and subsetting the NOTEEVENTS + ADMISSIONS datasets
# bert-large-uncased, bert-base-uncased, clinical-bert
!python preprocessing.py \
  --original_data "../mortality.csv" \
  --output_dir "../clinical-bert-data" \
  --temp_dir "../clinical-bert-tmp" \
  --task_name "FTLTrans_Readmission" \
  --log_path "../whatever.txt" \
  --id_num_neg 5276 \
  --id_num_pos 5276 \
  --random_seed 42 \
  --bert_model "./ClinicalBERT_pretraining"

New Pre-processing Job Start! 
original_data: ../mortality.csv, output_dir: ../clinical-bert-data, temp_dir: ../clinical-bert-tmp 
task_name: FTLTrans_Readmission, log_path: ../whatever.txt
id_num_neg: 5276, id_num_pos: 5276
random_seed: 42, bert_model: ./ClinicalBERT_pretraining
chunk 0 tokenize start!
First sentence tokenized
['normal', 'sin', '##us', 'rhythm', '.', 'non', '-', 'diagnostic', 'rep', '##olar', '##ization', 'abnormalities', '.', 'compared', 'to', 'the', '##pre', '##vious', 'tracing', 'of', 'no', 'major', 'change', ',', 'although', 'non', '-', 'diagnostic', 'rep', '##olar', '##ization', 'abnormalities', 'are', 'now', 'evident', '.', 'tracing', '#', '2']
chunk 1 tokenize start!
First sentence tokenized
['clinical', 'nutrition', 'o', ':', '~', '28', 'w', '##k', 'c', '##ga', 'bb', 'on', 'do', '##l', 'w', '##t', ':', '97', '##0', 'g', '(', '-', '60', ')', '(', '~', '25th', 'to', '50th', '%', 'ile', ')', ';', 'birth', 'w', '##t', ':', '124', '##5', 'g', '.', 'w', '##t', 'curr

In [ ]:
# Chunks the data
%cd ../
!python ./FTL-Trans/split_into_chunk.py \
  --data_dir "/content/drive/MyDrive/CS598-DL-Healthcare/clinical-bert-data" \
  --train_data "train.csv" \
  --val_data "val.csv" \
  --test_data "test.csv" \
  --log_path "log.txt" \
  --output_dir "clinical-bert-chunks" \
  --max_seq_length 128

/content/drive/My Drive/CS598-DL-Healthcare
New Split Job Start! 
data_dir: /content/drive/MyDrive/CS598-DL-Healthcare/clinical-bert-data, train_data: train.csv, val_data: val.csv, test_data: test.csv 
log_path: log.txt, output_dir: clinical-bert-chunks, max_seq_length: 128
100% 301257/301257 [00:38<00:00, 7904.87it/s]
100% 39035/39035 [00:04<00:00, 7818.53it/s]
100% 38635/38635 [00:04<00:00, 7772.62it/s]
In the train dataset Positive Patients' Chunks: 679334, Negative Patients' Chunks: 400818
In the validation dataset Positive Patients' Chunks: 89974, Negative Patients' Chunks: 53329
In the test dataset Positive Patients' Chunks: 86711, Negative Patients' Chunks: 54324
Split finished


In [ ]:
# Runs either of the models
#%cd ./FTL-Trans/
#!python3 ./run_clbert_lstm.py \
!python3 ./FTL.py \
  --data_dir "../clinical-bert-chunks" \
  --train_data "train.csv" \
  --val_data "val.csv" \
  --test_data "test.csv" \
  --log_path "./log.txt" \
  --bert_model "./ClinicalBERT_pretraining" \
  --output_dir "./LSTM-Clinical-Bert-Mod" \
  --embed_mode all \
  --task_name LSTM_Prediction \
  --max_seq_length    128 \
  --train_batch_size  32 \
  --eval_batch_size   1 \
  --learning_rate     2e-5 \
  --max_chunk_num     32 \
  --warmup_proportion 0.1 \
  --num_train_epochs  3 \
  --seed              42 \
  --gradient_accumulation_steps 1 \
  --save_model

in the modeling class
2025-05-05 13:35:05.907130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746452105.928288   25832 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746452105.934780   25832 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
05/05/2025 13:35:08 - INFO - modeling_readmission -   loading archive file ./ClinicalBERT_pretraining
05/05/2025 13:35:08 - INFO - modeling_readmission -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_h

In [ ]:
import time
import os
import torch
import random
import argparse
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from tqdm import trange
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# ←—— updated imports —————————————————————————————————————————————
from transformers import BertTokenizer, BertConfig
# ——————————————————————————————————————————————————————————————————————

from modeling_readmission import BertModel as ClinicalBertModel
from modeling_patient import FTLSTMLayer
from other_func import (
    write_log,
    Tokenize_with_note_id_hour,
    concat_by_id_list_with_note_chunk_id_time,
    convert_note_ids,
    flat_accuracy,
    write_performance,
    reorder_by_time,
)
from utils import time_batch_generator
from dotmap import DotMap
import matplotlib.pyplot as plt


In [ ]:
# Bert-Base-Uncased Performance
config = DotMap(
        hidden_dropout_prob=0.1,
        layer_norm_eps=1e-12,
        initializer_range=0.02,
        max_note_position_embedding=1000,
        max_chunk_position_embedding=1000,
        embed_mode="all",
        hidden_size=768,
        lstm_layers=1,
        task_name="test",
    )

test_df  = reorder_by_time(pd.read_csv("/content/drive/MyDrive/CS598-DL-Healthcare/bert-base-uncased-chunks/test.csv"))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
MAX_LEN = 128
el, ei, em, eni, et = Tokenize_with_note_id_hour(test_df,  MAX_LEN, tokenizer)
test_inputs, test_masks, test_labels, test_times = (
        torch.tensor(ei, dtype=torch.long),
        torch.tensor(em, dtype=torch.long),
        torch.tensor(el, dtype=torch.float),
        torch.tensor(et, dtype=torch.float),
    )
test_labels, test_inputs, test_masks, test_ids, test_note_ids, test_chunk_ids, test_times = \
      concat_by_id_list_with_note_chunk_id_time(test_df, el, test_inputs, test_masks,
                                                eni, test_times, MAX_LEN)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert = ClinicalBertModel.from_pretrained("bert-base-uncased").to(device)
lstm = FTLSTMLayer(config=config, num_labels=1).to(device)
ckpt = torch.load("./Bert-Base-Uncased/ckpt_epoch2.pt", map_location=device)
bert.load_state_dict(ckpt["bert_state"])
lstm.load_state_dict(ckpt["lstm_state"])
bert.eval(); lstm.eval()

val_acc = 0.0
val_steps = 0
gen_val = time_batch_generator(
    32, test_inputs, test_labels, test_masks,
        test_note_ids, test_chunk_ids, test_times
)
predictions, true_labels = [], []
for _ in range(len(test_ids)):
    with torch.no_grad():
        b_input_ids, b_labels, b_input_mask, b_note_ids, b_chunk_ids, b_times = next(gen_val)
        b_input_ids = b_input_ids.to(device)
        b_input_mask= b_input_mask.to(device)
        b_new_note_ids = convert_note_ids(b_note_ids).to(device)
        b_chunk_ids = b_chunk_ids.unsqueeze(0).to(device)
        b_times     = b_times.unsqueeze(0).to(device)
        b_labels = torch.tensor(b_labels, dtype=torch.float, device=device).unsqueeze(0)
        _, pooled_output = bert(
            b_input_ids, attention_mask=b_input_mask, token_type_ids=None
        )
        pooled_output = pooled_output.unsqueeze(0)
        preds = lstm(
            pooled_output, b_times, b_new_note_ids.unsqueeze(0), b_chunk_ids
        )  # second return = logits
        label_ids = b_labels.detach().cpu().numpy()[0]
        predictions.append(preds.detach().cpu().numpy())
        true_labels.append(label_ids)

    val_acc += flat_accuracy(preds.detach().cpu().numpy(),
                            b_labels.detach().cpu().numpy())
    val_steps += 1

# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_logits = [item for sublist in predictions for item in sublist]
flat_predictions = np.asarray([1 if i else 0 for i in (np.array(flat_logits) >= 0.5)])
flat_true_labels = np.asarray(true_labels)

from other_func import (
    accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef,
    model_auc, model_aupr
)
test_accuracy = accuracy_score(flat_true_labels, flat_predictions)
test_f1 = f1_score(flat_true_labels, flat_predictions, average='binary')
test_prec = precision_score(flat_true_labels, flat_predictions, average='binary')
test_rec = recall_score(flat_true_labels, flat_predictions, average='binary')
test_auc, _, _, _ = model_auc(flat_true_labels, flat_logits)
test_mc = matthews_corrcoef(flat_true_labels, flat_predictions)
test_aupr, _, _, _ = model_aupr(flat_true_labels, flat_logits)
print("Test Patient Level Accuracy: {}\n"
              "Test Patient Level F1 Score: {}\n"
              "Test Patient Level Precision: {}\n"
              "Test Patient Level Recall: {}\n"
              "Test Patient Level AUC: {} \n"
              "Test Patient Level Matthew's correlation coefficient: {}\n"
              "Test Patient Level AUPR: {} \n"
              "All Finished!".format(test_accuracy,
                                     test_f1,
                                     test_prec,
                                     test_rec,
                                     test_auc,
                                     test_mc,
                                     test_aupr))

Test Patient Level Accuracy: 0.7893738140417458
Test Patient Level F1 Score: 0.7819253438113949
Test Patient Level Precision: 0.8105906313645621
Test Patient Level Recall: 0.7552182163187856
Test Patient Level AUC: 0.8687209473983633 
Test Patient Level Matthew's correlation coefficient: 0.580102712061101
Test Patient Level AUPR: 0.8786625176824754 
All Finished!


In [ ]:
# Bert-Large-Uncased Performance
bert_cfg = BertConfig.from_pretrained("bert-large-uncased")
bert = ClinicalBertModel.from_pretrained("bert-large-uncased").to(device)
config = DotMap(
        hidden_dropout_prob=0.1,
        layer_norm_eps=1e-12,
        initializer_range=0.02,
        max_note_position_embedding=1000,
        max_chunk_position_embedding=1000,
        embed_mode="all",
        hidden_size=bert_cfg.hidden_size,
        lstm_layers=1,
        task_name="test",
    )

test_df  = reorder_by_time(pd.read_csv("/content/drive/MyDrive/CS598-DL-Healthcare/bert-large-uncased-chunks/test.csv"))
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased", do_lower_case=True)
MAX_LEN = 128
el, ei, em, eni, et = Tokenize_with_note_id_hour(test_df,  MAX_LEN, tokenizer)
test_inputs, test_masks, test_labels, test_times = (
        torch.tensor(ei, dtype=torch.long),
        torch.tensor(em, dtype=torch.long),
        torch.tensor(el, dtype=torch.float),
        torch.tensor(et, dtype=torch.float),
    )
test_labels, test_inputs, test_masks, test_ids, test_note_ids, test_chunk_ids, test_times = \
      concat_by_id_list_with_note_chunk_id_time(test_df, el, test_inputs, test_masks,
                                                eni, test_times, MAX_LEN)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert = ClinicalBertModel.from_pretrained("bert-large-uncased").to(device)
lstm = FTLSTMLayer(config=config, num_labels=1).to(device)
ckpt = torch.load("./Bert-Large-Uncased/ckpt_epoch2.pt", map_location=device)
bert.load_state_dict(ckpt["bert_state"])
lstm.load_state_dict(ckpt["lstm_state"])
bert.eval(); lstm.eval()

val_acc = 0.0
val_steps = 0
gen_val = time_batch_generator(
    32, test_inputs, test_labels, test_masks,
        test_note_ids, test_chunk_ids, test_times
)
predictions, true_labels = [], []
for _ in range(len(test_ids)):
    with torch.no_grad():
        b_input_ids, b_labels, b_input_mask, b_note_ids, b_chunk_ids, b_times = next(gen_val)
        b_input_ids = b_input_ids.to(device)
        b_input_mask= b_input_mask.to(device)
        b_new_note_ids = convert_note_ids(b_note_ids).to(device)
        b_chunk_ids = b_chunk_ids.unsqueeze(0).to(device)
        b_times     = b_times.unsqueeze(0).to(device)
        b_labels = torch.tensor(b_labels, dtype=torch.float, device=device).unsqueeze(0)
        _, pooled_output = bert(
            b_input_ids, attention_mask=b_input_mask, token_type_ids=None
        )
        pooled_output = pooled_output.unsqueeze(0)
        preds = lstm(
            pooled_output, b_times, b_new_note_ids.unsqueeze(0), b_chunk_ids
        )  # second return = logits
        label_ids = b_labels.detach().cpu().numpy()[0]
        predictions.append(preds.detach().cpu().numpy())
        true_labels.append(label_ids)

    val_acc += flat_accuracy(preds.detach().cpu().numpy(),
                            b_labels.detach().cpu().numpy())
    val_steps += 1

# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_logits = [item for sublist in predictions for item in sublist]
flat_predictions = np.asarray([1 if i else 0 for i in (np.array(flat_logits) >= 0.5)])
flat_true_labels = np.asarray(true_labels)

from other_func import (
    accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef,
    model_auc, model_aupr
)
test_accuracy = accuracy_score(flat_true_labels, flat_predictions)
test_f1 = f1_score(flat_true_labels, flat_predictions, average='binary')
test_prec = precision_score(flat_true_labels, flat_predictions, average='binary')
test_rec = recall_score(flat_true_labels, flat_predictions, average='binary')
test_auc, _, _, _ = model_auc(flat_true_labels, flat_logits)
test_mc = matthews_corrcoef(flat_true_labels, flat_predictions)
test_aupr, _, _, _ = model_aupr(flat_true_labels, flat_logits)
print("Test Patient Level Accuracy: {}\n"
              "Test Patient Level F1 Score: {}\n"
              "Test Patient Level Precision: {}\n"
              "Test Patient Level Recall: {}\n"
              "Test Patient Level AUC: {} \n"
              "Test Patient Level Matthew's correlation coefficient: {}\n"
              "Test Patient Level AUPR: {} \n"
              "All Finished!".format(test_accuracy,
                                     test_f1,
                                     test_prec,
                                     test_rec,
                                     test_auc,
                                     test_mc,
                                     test_aupr))

Test Patient Level Accuracy: 0.5455407969639469
Test Patient Level F1 Score: 0.5838401390095569
Test Patient Level Precision: 0.5384615384615384
Test Patient Level Recall: 0.6375711574952562
Test Patient Level AUC: 0.5744718772616472 
Test Patient Level Matthew's correlation coefficient: 0.09266478335988364
Test Patient Level AUPR: 0.5758099861445012 
All Finished!


In [ ]:
# Clinical-Bert Performance
bert_cfg = BertConfig.from_pretrained("./ClinicalBERT_pretraining")
bert = ClinicalBertModel.from_pretrained("./ClinicalBERT_pretraining").to(device)
config = DotMap(
        hidden_dropout_prob=0.1,
        layer_norm_eps=1e-12,
        initializer_range=0.02,
        max_note_position_embedding=1000,
        max_chunk_position_embedding=1000,
        embed_mode="all",
        hidden_size=bert_cfg.hidden_size,
        lstm_layers=1,
        task_name="test",
    )

test_df  = reorder_by_time(pd.read_csv("/content/drive/MyDrive/CS598-DL-Healthcare/clinical-bert-chunks/test.csv"))

tokenizer = BertTokenizer.from_pretrained("./ClinicalBERT_pretraining", do_lower_case=True)
MAX_LEN = 128
el, ei, em, eni, et = Tokenize_with_note_id_hour(test_df,  MAX_LEN, tokenizer)
test_inputs, test_masks, test_labels, test_times = (
        torch.tensor(ei, dtype=torch.long),
        torch.tensor(em, dtype=torch.long),
        torch.tensor(el, dtype=torch.float),
        torch.tensor(et, dtype=torch.float),
    )
test_labels, test_inputs, test_masks, test_ids, test_note_ids, test_chunk_ids, test_times = \
      concat_by_id_list_with_note_chunk_id_time(test_df, el, test_inputs, test_masks,
                                                eni, test_times, MAX_LEN)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert = ClinicalBertModel.from_pretrained("./ClinicalBERT_pretraining").to(device)
lstm = FTLSTMLayer(config=config, num_labels=1).to(device)

ckpt = torch.load("./Clinical-Bert-Mod/ckpt_epoch2.pt", map_location=device)
bert.load_state_dict(ckpt["bert_state"])
lstm.load_state_dict(ckpt["lstm_state"])
bert.eval(); lstm.eval()

val_acc = 0.0
val_steps = 0
gen_val = time_batch_generator(
    32, test_inputs, test_labels, test_masks,
        test_note_ids, test_chunk_ids, test_times
)
predictions, true_labels = [], []
for _ in range(len(test_ids)):
    with torch.no_grad():
        b_input_ids, b_labels, b_input_mask, b_note_ids, b_chunk_ids, b_times = next(gen_val)
        b_input_ids = b_input_ids.to(device)
        b_input_mask= b_input_mask.to(device)
        b_new_note_ids = convert_note_ids(b_note_ids).to(device)
        b_chunk_ids = b_chunk_ids.unsqueeze(0).to(device)
        b_times     = b_times.unsqueeze(0).to(device)
        b_labels = torch.tensor(b_labels, dtype=torch.float, device=device).unsqueeze(0)
        _, pooled_output = bert(
            b_input_ids, attention_mask=b_input_mask, token_type_ids=None
        )
        pooled_output = pooled_output.unsqueeze(0)
        preds = lstm(
            pooled_output, b_times, b_new_note_ids.unsqueeze(0), b_chunk_ids
        )  # second return = logits
        label_ids = b_labels.detach().cpu().numpy()[0]
        predictions.append(preds.detach().cpu().numpy())
        true_labels.append(label_ids)

    val_acc += flat_accuracy(preds.detach().cpu().numpy(),
                            b_labels.detach().cpu().numpy())
    val_steps += 1

# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_logits = [item for sublist in predictions for item in sublist]
flat_predictions = np.asarray([1 if i else 0 for i in (np.array(flat_logits) >= 0.5)])
flat_true_labels = np.asarray(true_labels)

from other_func import (
    accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef,
    model_auc, model_aupr
)
test_accuracy = accuracy_score(flat_true_labels, flat_predictions)
test_f1 = f1_score(flat_true_labels, flat_predictions, average='binary')
test_prec = precision_score(flat_true_labels, flat_predictions, average='binary')
test_rec = recall_score(flat_true_labels, flat_predictions, average='binary')
test_auc, _, _, _ = model_auc(flat_true_labels, flat_logits)
test_mc = matthews_corrcoef(flat_true_labels, flat_predictions)
test_aupr, _, _, _ = model_aupr(flat_true_labels, flat_logits)
print("Test Patient Level Accuracy: {}\n"
              "Test Patient Level F1 Score: {}\n"
              "Test Patient Level Precision: {}\n"
              "Test Patient Level Recall: {}\n"
              "Test Patient Level AUC: {} \n"
              "Test Patient Level Matthew's correlation coefficient: {}\n"
              "Test Patient Level AUPR: {} \n"
              "All Finished!".format(test_accuracy,
                                     test_f1,
                                     test_prec,
                                     test_rec,
                                     test_auc,
                                     test_mc,
                                     test_aupr))

Test Patient Level Accuracy: 0.793168880455408
Test Patient Level F1 Score: 0.781563126252505
Test Patient Level Precision: 0.8280254777070064
Test Patient Level Recall: 0.7400379506641366
Test Patient Level AUC: 0.8816760223095176 
Test Patient Level Matthew's correlation coefficient: 0.5896764022644368
Test Patient Level AUPR: 0.8873331184376569 
All Finished!


In [8]:
import time
import os
import torch
import random
import argparse
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from tqdm import trange
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# ←—— updated imports —————————————————————————————————————————————
from transformers import BertTokenizer, BertConfig
# ——————————————————————————————————————————————————————————————————————

from modeling_readmission import BertModel as ClinicalBertModel
from modeling_patient import LSTMLayer
from other_func import (
    write_log,
    Tokenize_with_note_id_hour,
    concat_by_id_list_with_note_chunk_id_time,
    convert_note_ids,
    flat_accuracy,
    write_performance,
    reorder_by_time,
)
from utils import time_batch_generator
from dotmap import DotMap
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

# Clinical-Bert Performance
bert_cfg = BertConfig.from_pretrained("./ClinicalBERT_pretraining")
bert = ClinicalBertModel.from_pretrained("./ClinicalBERT_pretraining").to(device)
config = DotMap(
        hidden_dropout_prob=0.1,
        layer_norm_eps=1e-12,
        initializer_range=0.02,
        max_note_position_embedding=1000,
        max_chunk_position_embedding=1000,
        embed_mode="all",
        hidden_size=bert_cfg.hidden_size,
        lstm_layers=1,
        task_name="test",
    )

test_df  = reorder_by_time(pd.read_csv("/content/drive/MyDrive/CS598-DL-Healthcare/clinical-bert-chunks/test.csv"))

tokenizer = BertTokenizer.from_pretrained("./ClinicalBERT_pretraining", do_lower_case=True)
MAX_LEN = 128
el, ei, em, eni, et = Tokenize_with_note_id_hour(test_df,  MAX_LEN, tokenizer)
test_inputs, test_masks, test_labels, test_times = (
        torch.tensor(ei, dtype=torch.long),
        torch.tensor(em, dtype=torch.long),
        torch.tensor(el, dtype=torch.float),
        torch.tensor(et, dtype=torch.float),
    )
test_labels, test_inputs, test_masks, test_ids, test_note_ids, test_chunk_ids, test_times = \
      concat_by_id_list_with_note_chunk_id_time(test_df, el, test_inputs, test_masks,
                                                eni, test_times, MAX_LEN)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert = ClinicalBertModel.from_pretrained("./ClinicalBERT_pretraining").to(device)
lstm = LSTMLayer(config=config, num_labels=1).to(device)

ckpt = torch.load("./LSTM-Clinical-Bert-Mod/ckpt_epoch2.pt", map_location=device)
bert.load_state_dict(ckpt["bert_state"])
lstm.load_state_dict(ckpt["lstm_state"])
bert.eval(); lstm.eval()

val_acc = 0.0
val_steps = 0
gen_val = time_batch_generator(
    32, test_inputs, test_labels, test_masks,
        test_note_ids, test_chunk_ids
)
predictions, true_labels = [], []
for _ in range(len(test_ids)):
    with torch.no_grad():
        b_input_ids, b_labels, b_input_mask, b_note_ids, b_chunk_ids = next(gen_val)
        b_input_ids = b_input_ids.to(device)
        b_input_mask= b_input_mask.to(device)
        b_new_note_ids = convert_note_ids(b_note_ids).to(device)
        b_chunk_ids = b_chunk_ids.unsqueeze(0).to(device)
        b_labels = torch.tensor(b_labels, dtype=torch.float, device=device).unsqueeze(0)
        _, pooled_output = bert(
            b_input_ids, attention_mask=b_input_mask, token_type_ids=None
        )
        pooled_output = pooled_output.unsqueeze(0)
        preds = lstm(
            pooled_output, b_new_note_ids.unsqueeze(0), b_chunk_ids
        )  # second return = logits
        label_ids = b_labels.detach().cpu().numpy()[0]
        predictions.append(preds.detach().cpu().numpy())
        true_labels.append(label_ids)

    val_acc += flat_accuracy(preds.detach().cpu().numpy(),
                            b_labels.detach().cpu().numpy())
    val_steps += 1

# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_logits = [item for sublist in predictions for item in sublist]
flat_predictions = np.asarray([1 if i else 0 for i in (np.array(flat_logits) >= 0.5)])
flat_true_labels = np.asarray(true_labels)

from other_func import (
    accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef,
    model_auc, model_aupr
)
test_accuracy = accuracy_score(flat_true_labels, flat_predictions)
test_f1 = f1_score(flat_true_labels, flat_predictions, average='binary')
test_prec = precision_score(flat_true_labels, flat_predictions, average='binary')
test_rec = recall_score(flat_true_labels, flat_predictions, average='binary')
test_auc, _, _, _ = model_auc(flat_true_labels, flat_logits)
test_mc = matthews_corrcoef(flat_true_labels, flat_predictions)
test_aupr, _, _, _ = model_aupr(flat_true_labels, flat_logits)
print("Test Patient Level Accuracy: {}\n"
              "Test Patient Level F1 Score: {}\n"
              "Test Patient Level Precision: {}\n"
              "Test Patient Level Recall: {}\n"
              "Test Patient Level AUC: {} \n"
              "Test Patient Level Matthew's correlation coefficient: {}\n"
              "Test Patient Level AUPR: {} \n"
              "All Finished!".format(test_accuracy,
                                     test_f1,
                                     test_prec,
                                     test_rec,
                                     test_auc,
                                     test_mc,
                                     test_aupr))

Test Patient Level Accuracy: 0.8889943074003795
Test Patient Level F1 Score: 0.8860759493670886
Test Patient Level Precision: 0.91
Test Patient Level Recall: 0.8633776091081594
Test Patient Level AUC: 0.9566375855600243 
Test Patient Level Matthew's correlation coefficient: 0.7790116852629192
Test Patient Level AUPR: 0.9586082350350303 
All Finished!
